In [1]:
from src.nanopore_dataset import BaseNanoporeDataset, custom_collate
from torch.utils.data import DataLoader
import jax
from jax import numpy as jnp

In [2]:
NON_RECURRENT_ENCODING_DICT = {'A':1, 'C':2, 'G':3, 'T':4, '':0}
NON_RECURRENT_DECODING_DICT = {1:'A', 2:'C', 3:'G', 4:'T', 0:''}
BASES = ['A', 'C', 'G', 'T']
BASES_CTC = 'N' + ''.join(BASES)

data_dir = "/Users/andrewtownsend/Desktop/SJSU/CS271/final_project/SimpleCall/practice_data/training_input"
batch_size = 64
s2s = False

In [3]:
dataset = BaseNanoporeDataset(
    data_dir = data_dir, 
    decoding_dict = NON_RECURRENT_DECODING_DICT, 
    encoding_dict = NON_RECURRENT_ENCODING_DICT, 
    split = 0.95, 
    shuffle = True, 
    seed = 1,
    s2s = s2s,
)

num train:  2
num valid:  1


/Users/andrewtownsend/.pyenv/versions/jax_flax_chex/lib/python3.11/site-packages/torch/utils/data/sampler.py:64: UserWarning: `data_source` argument is not used and will be removed in 2.2.0.You may still have custom implementation that utilizes it.
  warnings.warn("`data_source` argument is not used and will be removed in 2.2.0."


In [4]:
dataloader_train = DataLoader(
    dataset, 
    batch_size = batch_size, 
    sampler = dataset.train_sampler,
    collate_fn=custom_collate,
    num_workers = 1
)
dataloader_validation = DataLoader(
    dataset, 
    batch_size = batch_size, 
    sampler = dataset.validation_sampler,
    collate_fn=custom_collate,
    num_workers = 1
)

In [5]:
seed = 0
learning_rate = 1e-3
num_epochs = 5

In [6]:
from src.model import create_train_state, train_one_epoch, evaluate_model

In [7]:
train_state = create_train_state(key=jax.random.PRNGKey(seed), learning_rate=learning_rate)

In [8]:
test_metrics = evaluate_model(train_state, test_dataloader=dataloader_validation, alphabet=BASES_CTC)

batch num:  0


TypeError: Argument 'NACGT' of type <class 'str'> is not a valid JAX type

In [10]:
test_metrics["accuracy"]

Array(0.5, dtype=float32)

In [7]:

train_state = create_train_state(key=jax.random.PRNGKey(seed), learning_rate=learning_rate)

for epoch in range(1, num_epochs + 1):
    train_state, train_metrics, flag = train_one_epoch(train_state, dataloader_train, epoch)
    if flag:
        break
    print(f"Train epoch: {epoch}, loss: {train_metrics['loss']}, accuracy: {train_metrics['accuracy'] * 100}")

    # test_metrics = evaluate_model(train_state, test_inputs, test_targets)
    # print(f"\tTest epoch: {epoch}, loss: {test_metrics['loss']}, accuracy: {test_metrics['accuracy'] * 100}")

epoch:  1 , batch:  0
{'accuracy': Array(0.5, dtype=float32, weak_type=True), 'loss': Array(333.17853, dtype=float32)}
epoch:  1 , batch:  1
{'accuracy': Array(0.5, dtype=float32, weak_type=True), 'loss': Array(334.05536, dtype=float32)}
epoch:  1 , batch:  2
{'accuracy': Array(0.5, dtype=float32, weak_type=True), 'loss': Array(310.0108, dtype=float32)}
epoch:  1 , batch:  3
{'accuracy': Array(0.5, dtype=float32, weak_type=True), 'loss': Array(318.36694, dtype=float32)}
epoch:  1 , batch:  4
{'accuracy': Array(0.5, dtype=float32, weak_type=True), 'loss': Array(323.3166, dtype=float32)}
epoch:  1 , batch:  5
{'accuracy': Array(0.5, dtype=float32, weak_type=True), 'loss': Array(303.33563, dtype=float32)}
epoch:  1 , batch:  6
{'accuracy': Array(0.5, dtype=float32, weak_type=True), 'loss': Array(310.03143, dtype=float32)}
epoch:  1 , batch:  7
{'accuracy': Array(0.5, dtype=float32, weak_type=True), 'loss': Array(305.69177, dtype=float32)}
epoch:  1 , batch:  8
{'accuracy': Array(0.5, dtyp

KeyboardInterrupt: 